# Day 7

## Part 1

We need to run a program in chain, to compute the output of some amplifier.

The program take two input :
* a config value
* the previous output (starting from 0)

The interpreter for the program is the one we wrote on day 5.

In [1]:
from collections import defaultdict



def log(*args):
    if DEBUG:
        print(*args)


def get_param_value(program, position, param, mode):
    if mode == 0:  # position mode
        return program[program[position + param]]
    elif mode == 1:  # immediate mode
        return program[position + param]
    else:
        raise ValueError("Unknown paremeter mode")

        
def compute(program, read_input, write_output):
    """Compute the final state of a program, and return it.
    
    `program` is a list of int.
    `read_input` is a method returning a value each time it is called.
    `write_output` is a method taking a value as parameter.
    """
    program = program.copy()
    
    # init state
    p = 0
    
    # run program
    while True:
        # read instruction and split it into opcode and parameters mode
        
        try:
            instruction = program[p]
        except IndexError:
            # end of program
            break

        parameters_mode = defaultdict(lambda: 0)
        opcode = instruction % 100

        value = instruction // 100
        i = 1
        while value > 0:
            parameters_mode[i] = value % 10
            value //= 10
            i += 1
            
        log(p, '>', instruction, opcode, dict(parameters_mode))

        # execute opcode
        if opcode == 1:
            # add x y
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]

            result = x + y
            program[result_pos] = result
            log("add", x, y, result_pos, result)

            step = 4
        elif opcode == 2:
            # mult x y
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]
            
            result = x * y
            program[result_pos] = result
            log("mult", x, y, result_pos, result)

            step = 4
        elif opcode == 3:
            # read x
            result_pos = program[p + 1]

            value = read_input()
            program[result_pos] = value
            log("read", value, result_pos)

            step = 2
        elif opcode == 4:
            # write x
            value = get_param_value(program, p, 1, parameters_mode[1])
            write_output(value)
            log("write", value)
            
            step = 2
        elif opcode == 5:
            # jump-if-true x y
            value = get_param_value(program, p, 1, parameters_mode[1])
            jump_pos = get_param_value(program, p, 2, parameters_mode[2])
            
            log("jump-if-true", value, jump_pos)
            
            if value:
                p = jump_pos
                step = 0
            else:
                step = 3
        elif opcode == 6:
            # jump-if-false x y
            value = get_param_value(program, p, 1, parameters_mode[1])
            jump_pos = get_param_value(program, p, 2, parameters_mode[2])
            
            log("jump-if-false", value, jump_pos)
            
            if not value:
                p = jump_pos
                step = 0
            else:
                step = 3
        elif opcode == 7:
            # lt x y z
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]
            
            log("lt", x, y, result_pos)
            
            if x < y:
                program[result_pos] = 1
            else:
                program[result_pos] = 0
                
            step = 4
        elif opcode == 8:
            # eq x y z
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]
            
            log("eq", x, y, result_pos)
            
            if x == y:
                program[result_pos] = 1
            else:
                program[result_pos] = 0
                
            step = 4
        elif opcode == 99:
            # end of program
            log("stop")
            break
        else:
            # unknown instruction
            log("unknown opcode")
            break

        p += step

    return program

In [2]:
def compute_amplifier_ouput(config, program):
    """Compute the total output of some chained amplifiers.
    
    The program must take two input and output one value.
    The config is an array with the config value for each amplifier.
    """
    output = 0
    for config_value in config:
        result = []
        input_values = [output, config_value]
        read_input = lambda: input_values.pop()
        write_output = lambda v: result.append(v)

        compute(program, read_input, write_output)
        output = result[0]
        
    return output

This program should output 43210 with the config values 4, 3, 2, 1 and 0.

In [3]:
DEBUG = False
config = [4, 3, 2, 1, 0]
program = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
compute_amplifier_ouput(config, program)

43210

This one should output 54321.

In [4]:
config = [0, 1, 2, 3, 4]
program = [
    3,23,3,24,1002,24,10,24,1002,23,-1,23,
    101,5,23,23,1,24,23,23,4,23,99,0,0
]
compute_amplifier_ouput(config, program)

54321

This one should output 65210.

In [5]:
config = [1, 0, 4, 3, 2]
program = [
    3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,
    1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0
]
compute_amplifier_ouput(config, program)

65210

### Result

We want to find the *max output* the given program can output.
The config values can only range from 0 to 4.

In [6]:
from itertools import permutations

program = [3,8,1001,8,10,8,105,1,0,0,21,38,63,76,93,118,199,280,361,442,99999,3,9,101,3,9,9,102,3,9,9,101,4,9,9,4,9,99,3,9,1002,9,2,9,101,5,9,9,1002,9,5,9,101,5,9,9,1002,9,4,9,4,9,99,3,9,101,2,9,9,102,3,9,9,4,9,99,3,9,101,2,9,9,102,5,9,9,1001,9,5,9,4,9,99,3,9,102,4,9,9,1001,9,3,9,1002,9,5,9,101,2,9,9,1002,9,2,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,99]

max_output = 0
max_config = None
for config in permutations(range(5)):
    output = compute_amplifier_ouput(config, program)
    if output > max_output:
        max_output = output
        max_config = config
        
print(max_output)
print(max_config)

398674
(0, 3, 1, 2, 4)


## Part 2

Our ampfiliers are still no powerful enough, so we will use a feedback loop.
In this mode, the output of the 5th amplifier is connected back to the output of the first one.
The output flows over all amplifiers in a loop, and hopefully they all stop in the last iteration.
The output from the final iteration is the one we want to maximize.

The cavea is that the program for one amplifier must not be restarted between loops.
Instead the program will ask for as many input as loops are needed.

This means we need to tweek our interpreter in order to be able to run many program at once.
We will use async concurrency with [Trio](https://github.com/python-trio/trio) for that.

In [7]:
async def compute(program, read_input, write_output):
    """Compute the final state of a program, and return it.
    
    `program` is a list of int.
    `read_input` is a method returning a value each time it is called.
    `write_output` is a method taking a value as parameter.
    """
    program = program.copy()
    
    # init state
    p = 0
    
    # run program
    while True:
        # read instruction and split it into opcode and parameters mode
        
        try:
            instruction = program[p]
        except IndexError:
            # end of program
            break

        parameters_mode = defaultdict(lambda: 0)
        opcode = instruction % 100

        value = instruction // 100
        i = 1
        while value > 0:
            parameters_mode[i] = value % 10
            value //= 10
            i += 1
            
        log(p, '>', instruction, opcode, dict(parameters_mode))

        # execute opcode
        if opcode == 1:
            # add x y
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]

            result = x + y
            program[result_pos] = result
            log("add", x, y, result_pos, result)

            step = 4
        elif opcode == 2:
            # mult x y
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]
            
            result = x * y
            program[result_pos] = result
            log("mult", x, y, result_pos, result)

            step = 4
        elif opcode == 3:
            # read x
            result_pos = program[p + 1]

            value = await read_input()
            program[result_pos] = value
            log("read", value, result_pos)

            step = 2
        elif opcode == 4:
            # write x
            value = get_param_value(program, p, 1, parameters_mode[1])
            await write_output(value)
            log("write", value)
            
            step = 2
        elif opcode == 5:
            # jump-if-true x y
            value = get_param_value(program, p, 1, parameters_mode[1])
            jump_pos = get_param_value(program, p, 2, parameters_mode[2])
            
            log("jump-if-true", value, jump_pos)
            
            if value:
                p = jump_pos
                step = 0
            else:
                step = 3
        elif opcode == 6:
            # jump-if-false x y
            value = get_param_value(program, p, 1, parameters_mode[1])
            jump_pos = get_param_value(program, p, 2, parameters_mode[2])
            
            log("jump-if-false", value, jump_pos)
            
            if not value:
                p = jump_pos
                step = 0
            else:
                step = 3
        elif opcode == 7:
            # lt x y z
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]
            
            log("lt", x, y, result_pos)
            
            if x < y:
                program[result_pos] = 1
            else:
                program[result_pos] = 0
                
            step = 4
        elif opcode == 8:
            # eq x y z
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]
            
            log("eq", x, y, result_pos)
            
            if x == y:
                program[result_pos] = 1
            else:
                program[result_pos] = 0
                
            step = 4
        elif opcode == 99:
            # end of program
            log("stop")
            break
        else:
            # unknown instruction
            log("unknown opcode")
            break

        p += step

    return program

In [8]:
import trio

def read_input(queue):
    async def read_input_from_queue():
        value = await queue.receive()
        return value
    
    return read_input_from_queue


def write_output(queue):
    async def write_output_to_queue(value):
        return await queue.send(value)
    
    return write_output_to_queue


async def compute_amplifier_ouput(config, program):
    """Compute the total output of 5 chained amplifiers.
    
    The config is an array with the config value for each amplifier.
    The program must work in a feedback loop mode.
    """
    # create all queues
    
    # We connect the output from the 5th amplifier to the input from the 1st
    # amplifier.
    output5, input1 = trio.open_memory_channel(2)
    output1, input2 = trio.open_memory_channel(2)
    output2, input3 = trio.open_memory_channel(2)
    output3, input4 = trio.open_memory_channel(2)
    output4, input5 = trio.open_memory_channel(2)
    
    # prepare initial inputs

    # config values
    # We write in output5 the config value that will be read by the 1st
    # amplifier, in output1 the one for the 2d amplifier, and so on.
    await output5.send(config[0])
    await output1.send(config[1])
    await output2.send(config[2])
    await output3.send(config[3])
    await output4.send(config[4])
    # initual output value
    await output5.send(0)
    
    # run amplifiers
    async with trio.open_nursery() as nursery:
        nursery.start_soon(
            compute, program, read_input(input1), write_output(output1)
        )
        nursery.start_soon(
            compute, program, read_input(input2), write_output(output2)
        )
        nursery.start_soon(
            compute, program, read_input(input3), write_output(output3)
        )
        nursery.start_soon(
            compute, program, read_input(input4), write_output(output4)
        )
        nursery.start_soon(
            compute, program, read_input(input5), write_output(output5)
        )
    
    # There should be one value left on the 5th amplifier output, which is the
    # 1st one input
    return await input1.receive()

Ok, we are now stick to 5 amplifiers chain but it was for clarity.
Let's test this!

In [10]:
%autoawait trio

program = [
    3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
    27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5
]
config = [9, 8, 7, 6, 5]
await compute_amplifier_ouput(config, program)  # should return 139629729

139629729

In [11]:
%autoawait trio

program = [
    3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
    -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
    53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10
]
config = [9, 7, 8, 5, 6]
await compute_amplifier_ouput(config, program)  # should return 18216

18216

### Result

In [12]:
%autoawait trio

from itertools import permutations

program = [3,8,1001,8,10,8,105,1,0,0,21,38,63,76,93,118,199,280,361,442,99999,3,9,101,3,9,9,102,3,9,9,101,4,9,9,4,9,99,3,9,1002,9,2,9,101,5,9,9,1002,9,5,9,101,5,9,9,1002,9,4,9,4,9,99,3,9,101,2,9,9,102,3,9,9,4,9,99,3,9,101,2,9,9,102,5,9,9,1001,9,5,9,4,9,99,3,9,102,4,9,9,1001,9,3,9,1002,9,5,9,101,2,9,9,1002,9,2,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,99]

max_output = 0
max_config = None
for config in permutations(range(5, 10)):
    output = await compute_amplifier_ouput(config, program)
    if output > max_output:
        max_output = output
        max_config = config
        
print(max_output)
print(max_config)

39431233
(7, 8, 5, 9, 6)
